# Lab 9

**Aim:** To implement VGG CNN Architecture

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd
import torch.utils.data
import torchvision
import torchvision.datasets as dataset
import torchvision.transforms as T
import matplotlib.pyplot as plt
import numpy as np

from tqdm import tqdm
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [2]:
RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed_all(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

## Dataset

In [3]:
train_dataset = dataset.CIFAR100(root="./CIFAR100/train", train=True, transform=None, download=True)

100%|██████████| 169001437/169001437 [00:04<00:00, 40635766.53it/s]


Extracting ./CIFAR100/train/cifar-100-python.tar.gz to ./CIFAR100/train


In [4]:
x = np.concatenate([np.asarray(train_dataset[i][0]) for i in range(len(train_dataset))])
mean = np.mean(x, axis=(0, 1))/255
std = np.std(x, axis=(0,1))/255

mean = mean.tolist()
std = std.tolist()

print("mean:", mean)
print("std:", std)

mean: [0.5070751592371323, 0.48654887331495095, 0.4409178433670343]
std: [0.26733428587941854, 0.25643846292120615, 0.2761504713263903]


In [5]:
transform = T.Compose([T.ToTensor(),
                       T.Normalize(mean, std, inplace=True)])
train_dataset = dataset.CIFAR100(root="./CIFAR100/train", train=True, transform=transform, download=True)
valid_dataset = dataset.CIFAR100(root="./CIFAR100/val", train=True, transform=transform, download=True)
test_dataset = dataset.CIFAR100(root="./CIFAR100/test", train=False, transform=transform, download=True)

Files already downloaded and verified


100%|██████████| 169001437/169001437 [00:04<00:00, 41396572.41it/s]


Extracting ./CIFAR100/val/cifar-100-python.tar.gz to ./CIFAR100/val


100%|██████████| 169001437/169001437 [00:03<00:00, 43570322.81it/s]


Extracting ./CIFAR100/test/cifar-100-python.tar.gz to ./CIFAR100/test


In [6]:
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, num_workers=8)
valid_loader = DataLoader(valid_dataset, batch_size=256, shuffle=False, num_workers=8)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False, num_workers=8)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [7]:
images, labels = next(iter(train_loader))

In [8]:
print(images.shape)
print(labels.shape)
print(labels)

torch.Size([256, 3, 32, 32])
torch.Size([256])
tensor([27, 10, 73, 51, 53, 82, 21, 44, 47,  5, 94, 57, 17, 94, 92, 91, 53, 76,
        45, 46, 52, 30, 30, 85, 39, 70,  2,  6, 30, 58, 36, 67, 61, 38, 28, 10,
        61, 46, 43, 61, 27, 22, 86, 11, 43, 87, 78, 47, 97,  2, 94, 16, 79, 92,
        84,  8, 91, 35, 61, 55, 75, 42, 60, 49, 41, 96, 46, 97, 37, 34, 58,  4,
        89, 51, 92, 62, 66, 40, 95, 26, 31, 86, 33, 82, 58, 14, 68, 26, 96, 23,
        87, 82, 15, 65, 26,  3,  1, 92, 60, 32, 16, 44, 20, 86, 36, 56, 99, 61,
        72, 46, 91, 84, 89, 30, 37,  0, 69, 68, 58, 11, 46, 41, 98, 46, 15,  2,
        21, 37, 59, 79, 88, 71, 65, 70, 60, 21, 62, 92, 40, 29,  6, 48, 40, 66,
        90, 98, 44, 64, 46, 13,  6,  6, 18, 80, 33, 76, 27, 45, 11, 61, 59, 96,
        60, 25, 80, 66, 48, 87, 24,  1, 35, 56,  2, 30,  3,  7, 29, 47, 81, 23,
         2, 91, 12,  3, 40, 15, 70, 68, 25,  3, 31, 73, 10, 87, 85, 45, 74, 58,
         7, 56, 20, 84, 93, 34, 15, 45,  4, 82, 61, 42, 10, 82, 91, 46, 5

## Model

In [9]:

def conv_block(in_channels, out_channels):
    return nn.Sequential(nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1),
                         nn.BatchNorm2d(out_channels),
                         nn.ReLU())

class VGG19(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels

        self.conv_block1 = nn.Sequential(nn.Conv2d(in_channels=self.in_channels, out_channels=64, kernel_size=3, stride=1, padding=1),
                                         nn.BatchNorm2d(64),
                                         nn.ReLU(),
                                         conv_block(64, 64),
                                         nn.MaxPool2d(kernel_size=2, stride=2))

        self.conv_block2 = nn.Sequential(nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
                                         nn.BatchNorm2d(128),
                                         nn.ReLU(),
                                         conv_block(128, 128),
                                         nn.MaxPool2d(kernel_size=2, stride=2))

        self.conv_block3 = nn.Sequential(nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1),
                                         nn.BatchNorm2d(256),
                                         nn.ReLU(),
                                         *[conv_block(256, 256) for _ in range(3)],
                                         nn.MaxPool2d(kernel_size=2, stride=2))

        self.conv_block4 = nn.Sequential(nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1),
                                         nn.BatchNorm2d(512),
                                         nn.ReLU(),
                                         *[conv_block(512, 512) for _ in range(3)],
                                         nn.MaxPool2d(kernel_size=2, stride=2))

        self.conv_block5 = nn.Sequential(*[conv_block(512, 512) for _ in range(4)],
                                         nn.MaxPool2d(kernel_size=2, stride=2))

        self.avg_pool = nn.AdaptiveAvgPool2d(output_size=(7,7))

        self.linear1 = nn.Sequential(nn.Linear(in_features=7*7*512, out_features=4096, bias=True),
                                     nn.Dropout(0.5),
                                     nn.ReLU())
        self.linear2 = nn.Sequential(nn.Linear(in_features=4096, out_features=4096, bias=True),
                                     nn.Dropout(0.5),
                                     nn.ReLU())
        self.linear3 = nn.Linear(in_features=4096, out_features=self.out_channels, bias=True)

    def forward(self, x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.conv_block3(x)
        x = self.conv_block4(x)
        x = self.conv_block5(x)
        x = self.avg_pool(x)
        x = self.linear1(x.view(x.shape[0], -1))
        x = self.linear2(x)
        x = self.linear3(x)
        return x

## Train

In [10]:
model = VGG19(in_channels=3, out_channels=100)
optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=0.005)
criterion = nn.CrossEntropyLoss()
total_epochs = 50
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)

In [11]:
def train(model, optimizer, train_loader, val_loader, criterion, total_epochs):
    print("Training Begin!")
    print()
    best_accuracy = 0

    for epoch in range(total_epochs):
        model.train()

        for step, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)

            optimizer.zero_grad(set_to_none=True)
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            if (step+1) % 30 == 0:
              print('Epoch: [{}/{}] | Step: [{}/{}] | Loss: {:.4f}'.format(epoch+1, total_epochs, step+1, len(train_loader), loss.item()))

        with torch.no_grad():
            print("Validating...")
            model.eval()
            total = 0
            correct = 0
            for (images, labels) in val_loader:
                images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)

                outputs = model(images)
                _, predicted = torch.max(outputs, dim=1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

            acc = (correct/total)*100
            if acc > best_accuracy:
                best_accuracy = acc
                torch.save(model.state_dict(), "./model_cbam2.pt")
            print(f"Current Accuracy: {acc:.3f}%")
            print(f"Best Accuracy: {best_accuracy:.3f}%")
            print()

    print('Train Finished!')

In [12]:
train(model=model, optimizer=optimizer, train_loader=train_loader, val_loader=valid_loader, criterion=criterion, total_epochs=total_epochs)

Training Begin!

Epoch: [1/50] | Step: [30/196] | Loss: 4.5930
Epoch: [1/50] | Step: [60/196] | Loss: 4.5581
Epoch: [1/50] | Step: [90/196] | Loss: 4.3560
Epoch: [1/50] | Step: [120/196] | Loss: 4.3529
Epoch: [1/50] | Step: [150/196] | Loss: 4.2601
Epoch: [1/50] | Step: [180/196] | Loss: 4.2955
Validating...
Current Accuracy: 3.092%
Best Accuracy: 3.092%

Epoch: [2/50] | Step: [30/196] | Loss: 4.1185
Epoch: [2/50] | Step: [60/196] | Loss: 4.1673
Epoch: [2/50] | Step: [90/196] | Loss: 3.9969
Epoch: [2/50] | Step: [120/196] | Loss: 3.9640
Epoch: [2/50] | Step: [150/196] | Loss: 3.8409
Epoch: [2/50] | Step: [180/196] | Loss: 3.7740
Validating...
Current Accuracy: 7.658%
Best Accuracy: 7.658%

Epoch: [3/50] | Step: [30/196] | Loss: 3.7485
Epoch: [3/50] | Step: [60/196] | Loss: 3.7498
Epoch: [3/50] | Step: [90/196] | Loss: 3.4678
Epoch: [3/50] | Step: [120/196] | Loss: 3.5256
Epoch: [3/50] | Step: [150/196] | Loss: 3.4212
Epoch: [3/50] | Step: [180/196] | Loss: 3.5045
Validating...
Current 